In [15]:
import networkx as nx
import random as r
import operator

#input graph G and number of desired districts k
#outputs a k partition of G respresented as a list of lists
def agglom(G,k):    
    for n in (G.nodes()):  #initializes node attributes which track contractions
        G.node[n]['seg'] = [n]
    edges = {}
    for e in (list(set(G.edges()) | set(nx.non_edges(G)))):  #initializes edge attributes which track 
        edges[e] = 1                                         #edge contractions, which informs edge weighting
        
    while len(G.nodes) > k and len(G.edges) > 0 : #iterates until desired districts are produced or there's no more edges
        weight = 0
        while r.uniform(0,1) > weight:            #rejection sampling for edge weighting
            e = list(G.edges())[r.randint(0,len(G.edges)-1)]
            weight = 1/edges[e]                                       #####edge contraction is weighted negatively. reciprocal
        inter = list(set(G.neighbors(e[0])) & set(G.neighbors(e[1]))) ##### weighting grows too slowly for large graphs so other 
        for shared in inter:                                          ##### weighting strategies are suggested such as .9**(edges[e]**5 - 1)
            if shared > e[1] : edges[(e[0],shared)] += edges[(e[1],shared)] 
            elif shared < e[0]: edges[(shared,e[0])] += edges[(shared,e[1])] #recording edge contraction
            else : edges[(e[0],shared)] += edges[(shared,e[1])]
        G.node[e[0]]['seg'] += (G.node[e[1]]['seg'])      #recording node contraction
        G = nx.contracted_edge(G, e, self_loops = False)  #contraction occurs
    result = nx.get_node_attributes(G,'seg')
    out = list()
    for w in result:
        out.append(sorted(result[w]))  #formatting output
    return out

In [16]:
#Example: 
G = nx.Graph()
G.add_nodes_from(range(0,5)) #add nodes
G.add_edges_from(nx.non_edges(G))
print (agglom(G,2))

[[0, 1, 2], [3, 4]]
